In [89]:
from llama_cpp import Llama
import re
from asteval import Interpreter
import math

In [90]:
llm = Llama.from_pretrained(
	repo_id="Qwen/Qwen1.5-1.8B-Chat-GGUF",
	filename="qwen1_5-1_8b-chat-q4_k_m.gguf",
    verbose=False,
)
     
aeval = Interpreter()
aeval.symtable['math'] = math

llama_context: n_ctx_per_seq (512) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


In [95]:
def run_llama(user_input: str) -> str:
    system_message = {
        "role": "system",
        "content": (
            "You are a helpful AI assistant.\n"
            "When a question requires calculation, DO NOT solve it yourself or show any steps.\n"
            "Respond ONLY with this exact format:\n"
            "USE_CALCULATOR: <valid Python expression>\n\n"
            "Do NOT write anything else, no explanations, no intermediate steps, no equal signs.\n"
            "Use Python's math module for math functions like sqrt, sin, cos, radians, etc. ONLY WHEN NECESSARY.\n" 
            "For regular arithmatic use Python's built-in operators like +, -, *, /, //, %, **.\n\n"
            "For non-math questions, answer normally.\n\n"
            "Examples:\n"
            "Q: What is 3 + 4?\n"
            "A: USE_CALCULATOR: 3 + 4\n\n"
            "Q: What is the capital of France?\n"
            "A: The capital of France is Paris.\n\n"
            "Q: What is the sine of 30 degrees?\n"
            "A: USE_CALCULATOR: math.sin(math.radians(30))\n"
            "Q: What is 15 times 34554 divided by 27?\n"
            "A: USE_CALCULATOR: 15 * 34554 / 27\n"
            "Do not try to solve the math problems yourself, just return the expression.\n"
        )
    }

    user_message = {
        "role": "user",
        "content": f"Q: {user_input.strip()}\nA:"
    }

    messages = [system_message, user_message]

    response = llm.create_chat_completion(
        messages=messages,
        max_tokens=100,
        temperature=0.0,
    )

    raw_text = response['choices'][0]['message']['content']

    return raw_text.strip()

In [96]:
def eval_expression(expression: str) -> str:
    print(f"Evaluating expression: {expression}")
    print("-" * 20)
    result = aeval(expression)

    if aeval.error:
        print(f"Error evaluating expression: {aeval.error}")
        print("-" * 20)
        return f"Error: {aeval.error}"
    
    return result

def parse_response(response):
    print(f"Raw response: {response}")
    print("-" * 20)
    match = re.match(r"USE_CALCULATOR:\s*(.+)", response)
    if match:
        expression = match.group(1)
        return f"Result: {eval_expression(expression)}"
    return response


In [98]:
result = run_llama("What is the sine of 45 degrees multiplied by 1234.56 divided by 7.89 plus the logarithm base 2 of 256 minus the square root of 1024 plus 5 cubed divided by 2 times the factorial of 4?")
# result = run_llama("what is the capital of France")

print(parse_response(result))

Raw response: USE_CALCULATOR: math.sin(45) * math.pow(1234.56, 7.89) + math.log2(256) - math.sqrt(1024) + 5 ** (4/2) / 2 * math.factorial(4)
Note: The expression includes several mathematical operations that involve trigonometric functions, logarithms, and factorials. Here's a breakdown of the calculations:

1. The
--------------------
Evaluating expression: math.sin(45) * math.pow(1234.56, 7.89) + math.log2(256) - math.sqrt(1024) + 5 ** (4/2) / 2 * math.factorial(4)
--------------------
Result: 2.0985126590906684e+24
